In [1]:
%load_ext autoreload
%autoreload 2

from __future__ import print_function, division
import os

os.environ["CUDA_VISIBLE_DEVICES"] = '7'

In [2]:
import ultralytics
from IPython.display import display, Image
from roboflow import Roboflow
import cv2
import sys
import numpy as np
import matplotlib.pyplot as plt

import torch

# Check Ultralytics version and setup completion
ultralytics.checks()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

from ultralytics import YOLO

Ultralytics YOLOv8.0.137 🚀 Python-3.9.16 torch-2.1.0.dev20230314 CUDA:0 (NVIDIA A100-SXM4-80GB, 81100MiB)
Setup complete ✅ (128 CPUs, 1007.5 GB RAM, 35.5/48.9 GB disk)


In [71]:
im_path = '/home/zt16/code/priv-nerf/nerf_pl/data/brandenburg_gate/dense/images/50554919_2069783324.jpg'

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Perform object detection on the image
results = model.predict(source=im_path, conf=0.25)


image 1/1 /home/zt16/code/priv-nerf/nerf_pl/data/brandenburg_gate/dense/images/50554919_2069783324.jpg: 480x640 (no detections), 7.1ms
Speed: 1.9ms preprocess, 7.1ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)


In [73]:
len(results[0])

0

In [42]:
image = cv2.imread(im_path, cv2.IMREAD_COLOR)

from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
sam_checkpoint = "/home/zt16/code/priv-nerf/nerfw_pl_priv/sam_ckpt/sam_vit_h_4b8939.pth"
model_type = "vit_h"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
predictor = SamPredictor(sam)
predictor.set_image(image)

In [43]:
bbox = []
for boxes in results[0].boxes:
#     import pdb; pdb.set_trace()
    if boxes.cls[0] == 0.:
        bbox.append(boxes.xyxy.tolist()[0])
        
      

In [44]:
predicted_boxes = results[0].boxes.xyxy
transformed_boxes = predictor.transform.apply_boxes_torch(predicted_boxes, image.shape[:2])

In [45]:
predicted_boxes_np=predicted_boxes.cpu().numpy()

In [46]:
# bbox_arr = np.array(bbox)
masks, _, _ = predictor.predict_torch(
    point_coords=None,
    point_labels=None,
    boxes=transformed_boxes,
    multimask_output=False,
)


In [49]:
masks = np.array(masks.cpu())
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

(15, 1, 767, 1023)

In [60]:

final_mask = None
for i in range(len(masks) - 1):
  if final_mask is None:
    final_mask = np.bitwise_or(masks[i][0], masks[i+1][0])
  else:
    final_mask = np.bitwise_or(final_mask, masks[i+1][0])

final_mask = np.bitwise_not(final_mask)
mask_rgb = np.stack((final_mask,final_mask, final_mask), axis=-1)
result = np.multiply(image_rgb, mask_rgb)

plt.figure(figsize=(10, 10))
# plt.imshow(image_rgb)
plt.axis('off')
plt.imshow(result)

plt.show()

In [63]:
result.shape

(767, 1023, 3)

In [75]:
input_folder = '/home/zt16/code/priv-nerf/nerf_pl/data/brandenburg_gate/dense/images'
output_folder = '/home/zt16/code/priv-nerf/nerf_pl/data/brandenburg_gate/dense/masked_images'
    # Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
# Get a list of all files in the input folder
files = os.listdir(input_folder)
import pdb; pdb.set_trace
for file in files:
    print(file)
    im_path = os.path.join(input_folder, file)
    output_path = os.path.join(output_folder, file)
    
    image = cv2.imread(im_path, cv2.IMREAD_COLOR)
#     image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform object detection on the image
    results = model.predict(source=im_path, conf=0.25)
    if len(results[0]) == 0:        
        cv2.imwrite(output_path, image)
        continue
        
        
    
    
    predicted_boxes = results[0].boxes.xyxy
    transformed_boxes = predictor.transform.apply_boxes_torch(predicted_boxes, image.shape[:2])

    predictor.set_image(image)

    masks, _, _ = predictor.predict_torch(
        point_coords=None,
        point_labels=None,
        boxes=transformed_boxes,
        multimask_output=False,
        )
    masks = np.array(masks.cpu())
    
    final_mask = None
    
    for i in range(len(masks) - 1):
      if final_mask is None:
        final_mask = np.bitwise_or(masks[i][0], masks[i+1][0])
      else:
        final_mask = np.bitwise_or(final_mask, masks[i+1][0])
    if final_mask is None:
        result = image
    else:
        final_mask = np.bitwise_not(final_mask)
        mask_rgb = np.stack((final_mask,final_mask, final_mask), axis=-1)
        result = np.multiply(image, mask_rgb)

    
    output_path = os.path.join(output_folder, file)
    cv2.imwrite(output_path, result)

        
        
        
        
        
        
    


image 1/1 /home/zt16/code/priv-nerf/nerf_pl/data/brandenburg_gate/dense/images/60633305_3067485286.jpg: 384x640 12 persons, 1 bicycle, 2 cars, 6.8ms
Speed: 1.5ms preprocess, 6.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)


60633305_3067485286.jpg



image 1/1 /home/zt16/code/priv-nerf/nerf_pl/data/brandenburg_gate/dense/images/71180317_4509734040.jpg: 448x640 1 person, 6.2ms
Speed: 1.6ms preprocess, 6.2ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)


71180317_4509734040.jpg



image 1/1 /home/zt16/code/priv-nerf/nerf_pl/data/brandenburg_gate/dense/images/27679520_326509985.jpg: 480x640 12 persons, 5.5ms
Speed: 1.4ms preprocess, 5.5ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)


...

76577092_6779376230.jpg



image 1/1 /home/zt16/code/priv-nerf/nerf_pl/data/brandenburg_gate/dense/images/44883739_4514141521.jpg: 640x480 12 persons, 1 car, 5.7ms
Speed: 1.5ms preprocess, 5.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 480)


44883739_4514141521.jpg



image 1/1 /home/zt16/code/priv-nerf/nerf_pl/data/brandenburg_gate/dense/images/54566619_7723106684.jpg: 640x448 1 bird, 1 clock, 6.4ms
Speed: 1.9ms preprocess, 6.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 448)


54566619_7723106684.jpg


In [11]:
def mask_entire_folders(input_folder, output_folder):
    mask_images(input_folder,output_folder)

(767, 1023, 3)


In [19]:
print(boxes.cls[0])

tensor(0., device='cuda:0')
